# Lab05: 2.5 GHz LPF Design

Due: Febuary 26, 2019

Author: Kevin Egedy

## Objective

Designing, fabricating, and testing a microwave low pass filter using the insertion loss method and stepped impedance method.

## Design Goals
- Cutoff freq is 2.5GHz
- Maximally flat response
- Minimum attenuation of 25 dB at 5GHz
- Minimum TL width is 0.4mm
- 10 mm section of 50 ohm TL on both ends

In [1]:
# Libraries
#
import sympy as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as pch
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Notes

<img src="images/Butterworth_Attenuation.png" width="600">

<img src="images/Butterworth_ElementValues.png" width="600">

<img src="images/Kuroda_Identities.png" width="600">

**Equations**

$\begin{eqnarray}
L_k' &=& \frac{R_0 L_k}{\omega_c}  \\
C_k' &=& \frac{C_k}{R_0 \omega_c} \\
Z_{\text{in}} &\rightarrow& 
              \text{SC}_{(l=\frac{\lambda}{8})} &=&
              -jZ_1 \tan{\beta l} &=& 
              jZ_1 &=&  jL \\
Z_{\text{in}} &\rightarrow& 
              \text{OC}_{(l=\frac{\lambda}{8})} &=&
              -jZ_1 \cot{\beta l} &=&
               -jZ_1 &=& \frac{1}{jC}
\end{eqnarray}$

In [2]:
def Lk_prime (Lk, R0, wc): 
    return R0*Lk/wc
def Ck_prime (Ck, R0, wc): 
    return Ck/(R0*wc)

R0 = 50
f0 = 2.5*10**9
wc = 2*np.pi*f0
print(f'L1 = {Lk_prime(0.7654,R0,wc)}')
print(f'C1 = {Ck_prime(1.8478,R0,wc)}')
print(f'L2 = {Lk_prime(1.8478,R0,wc)}')
print(f'C2 = {Ck_prime(0.7654,R0,wc)}')

L1 = 2.436343868850734e-09
C1 = 2.3526920307616338e-12
L2 = 5.881730076904084e-09
C2 = 9.745375475402936e-13


### Butterworth Filter Via Insertion Loss Method


#### Ideal TL Model

In order to achieve $25$dB at $5$GHz, need at least $4$ reactive elements (obtained from attenuation table). In my solution, I choose $N=4$ elements.

| Parameter  | Element  |  Normalized Value ($\omega_c$=1) | Circuit Value ($Z_0=50$) |
|:----------:|:-----:|:------:|:---------:|
| $g_1$      | $L_1$ | 0.7654 | 2.4363 nH |
| $g_2$      | $C_1$ | 1.8478 | 2.3527 pF |
| $g_3$      | $L_2$ | 1.8478 | 5.8817 nH |
| $g_4$      | $C_1$ | 0.7654 | 0.9745 pF |
| $g_5$      | $R_L$ | 1.0000 | 50.00 Ohm |

<img src="images/Kuroda_Model.png" width="400">

**Kuroda Identities**

<img src="images/Kuroda_CaseA_1.png" width="400">

In [3]:
Z1,Z2,n_sqr = sp.symbols('Z1,Z2,n_sqr')
system = sp.Matrix([
    [Z1/n_sqr-0.765],
    [Z2/n_sqr-1],
    [1+Z2/Z1-n_sqr]
])
sp.solve(system)

[{Z1: 1.76500000000000, Z2: 2.30718954248366, n_sqr: 2.30718954248366}]

<img src="images/Kuroda_CaseA_2.png" width="400">

In [4]:
Z1 = 1
Z2 = 1.307
n_sqr = 1+Z2/Z1
Z1/n_sqr,Z2/n_sqr

(0.43346337234503685, 0.5665366276549632)

**Regroup**

<img src="images/Kuroda_Summary1.png" width="400">

**Solve Remaining Series TL**

<img src="images/Kuroda_CaseB_1.png" width="500">

In [10]:
Z1 = 1.848
Z2 = 0.567
n_sqr = 1+Z2/Z1
Z1*n_sqr,Z2*n_sqr

(2.415, 0.740965909090909)

<img src="images/Kuroda_CaseB_2.png" width="450">

In [9]:
Z1 = 0.433
Z2 = 1
n_sqr = 1+Z2/Z1
Z1*n_sqr,Z2*n_sqr

(1.4329999999999998, 3.3094688221709005)

**Normalized**
<img src="images/Butterworth_Ideal_Summary.png" width="450">


**50 Ohm**
<img src="images/Butterworth_Ideal_Summary_Actual.png" width="450">

**Ideal Simulation**
<img src="images/Butterworth_Ideal_S11.png" width="450">
<img src="images/Butterworth_Ideal_S21.png" width="450">

### Stepped Impedance Filter

Requirements:
- $Z_{\text{high}} = 150 \Omega$
- $Z_{\text{low}} = 15 \Omega$

<img src="images/Stepped_Impedance_Model.png" width="450">

**Short high-$Z_0$ TL**

- $\beta l = \frac{LR_0}{Z_h} = \frac{1 \cdot 50}{150} = 0.33\text{radians} = {19.1}^{\circ} $

**Short low-$Z_0$ TL**

- $\beta l = \frac{C}{R_0}Z_l = \frac{2 \cdot 15}{50} = 0.6\text{radians} = {34.38}^{\circ} $

**Ideal TL Model**

<img src="images/Stepped_Impedance_Ideal_TL.png" width="450">

**Ideal Simulation**

<img src="images/Stepped_Ideal_S11.png" width="450">
<img src="images/Stepped_Ideal_S21.png" width="450">